# Predcting housing prices with neural networks

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import sys
sys.path.insert(0,'..')
from UnsupervisedLearning import Stochastic_K_means
from MultilayerPerceptronNetwork import Train_MLP, Test_MLP
import nn_Utilities as nn

In [26]:
# Data read and import
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) 
validationData = pd.read_csv('test.csv')
trainData = pd.read_csv('train.csv')

print(trainData.columns)

Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'Alley', u'LotShape', u'LandContour', u'Utilities',
       u'LotConfig', u'LandSlope', u'Neighborhood', u'Condition1',
       u'Condition2', u'BldgType', u'HouseStyle', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'RoofStyle',
       u'RoofMatl', u'Exterior1st', u'Exterior2nd', u'MasVnrType',
       u'MasVnrArea', u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1',
       u'BsmtFinType2', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF',
       u'Heating', u'HeatingQC', u'CentralAir', u'Electrical', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'KitchenQual', u'TotRmsAbvGrd', u'Functional',
       u'Fireplaces', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish',

In [5]:
from sklearn.linear_model import LinearRegression

In [6]:
reg = LinearRegression()
labels = trainData['SalePrice']
conv_dates = [1 if values == 2010 else 0 for values in trainData.YrSold ]
trainData['YrSold'] = conv_dates


dropList = ['Id', 'SalePrice']
for e in trainData.columns:
    if trainData[e].dtype == object:
        dropList.append(e)
        
#train1 = trainData.loc[:,trainData.dtypes != object]
train1 = trainData.drop(dropList, axis = 1)
train1 = train1.astype(np.float64, copy=False)
print(train1.columns)
print(len(dropList))

Index([u'MSSubClass', u'LotFrontage', u'LotArea', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'MasVnrArea',
       u'BsmtFinSF1', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'TotRmsAbvGrd', u'Fireplaces', u'GarageYrBlt',
       u'GarageCars', u'GarageArea', u'WoodDeckSF', u'OpenPorchSF',
       u'EnclosedPorch', u'3SsnPorch', u'ScreenPorch', u'PoolArea', u'MiscVal',
       u'MoSold', u'YrSold'],
      dtype='object')
45


In [9]:
from sklearn.cross_validation import train_test_split

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
train1.isnull = 0
labels.isnull = 0
dropList1 = []
for e in train1.columns:
    if np.isnan(train1[e]).any():
        dropList1.append(e)
print(len(dropList1))
train1 = train1.drop(dropList1, axis = 1)

train1.head()

3


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60.000,8450.000,7.000,5.000,2003.000,2003.000,706.000,0.000,150.000,856.000,...,548.000,0.000,61.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000
1,20.000,9600.000,6.000,8.000,1976.000,1976.000,978.000,0.000,284.000,1262.000,...,460.000,298.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000
2,60.000,11250.000,7.000,5.000,2001.000,2002.000,486.000,0.000,434.000,920.000,...,608.000,0.000,42.000,0.000,0.000,0.000,0.000,0.000,9.000,0.000
3,70.000,9550.000,7.000,5.000,1915.000,1970.000,216.000,0.000,540.000,756.000,...,642.000,0.000,35.000,272.000,0.000,0.000,0.000,0.000,2.000,0.000
4,60.000,14260.000,8.000,5.000,2000.000,2000.000,655.000,0.000,490.000,1145.000,...,836.000,192.000,84.000,0.000,0.000,0.000,0.000,0.000,12.000,0.000


In [10]:
x_train,x_test,y_train,y_test = train_test_split(train1,labels,test_size = 0.1, random_state = 5)

In [11]:
reg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [12]:
reg.score(x_test,y_test)

0.80947268501983927

In [13]:
from sklearn import ensemble

In [14]:
clf = ensemble.GradientBoostingRegressor(n_estimators= 400, max_depth=5,
                                         min_samples_split=2, learning_rate= 0.01, loss = 'ls')


In [18]:
clf.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=400, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [19]:
clf.score(x_test, y_test)

0.87547966058047688

In [20]:
from sklearn import neural_network

In [21]:
MLP_Reg = neural_network.MLPRegressor(activation = 'relu',solver='adam', random_state=2 , max_iter=10**6)            

In [24]:
MLP_Reg.fit(x_train, y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=2, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [25]:
MLP_Reg.score(x_test, y_test)

0.77180878797466079